In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('export.csv')
df.head()

,DPT_EXE,DEPART,ARRIVEE,NBRE
0,71,71528,69123,1
1,21,21142,21317,1
2,75,91350,91000,1
3,21,21663,71076,1
4,21,21304,75113,1


In [4]:
depart_freq = df['DEPART'].value_counts().idxmax()
arrivee_freq = df['ARRIVEE'].value_counts().idxmax()

print(f"Code postal le plus fréquent pour DEPART : {depart_freq}")
print(f"Code postal le plus fréquent pour ARRIVEE : {arrivee_freq}")

Code postal le plus fréquent pour DEPART : 21231
Code postal le plus fréquent pour ARRIVEE : 21231


In [5]:
depart_top_10 = df['DEPART'].value_counts().head(10)
arrivee_top_10 = df['ARRIVEE'].value_counts().head(10)

print("Top 10 codes postaux les plus fréquents pour DEPART :")
print(depart_top_10)

print("\nTop 10 codes postaux les plus fréquents pour ARRIVEE :")
print(arrivee_top_10)

Top 10 codes postaux les plus fréquents pour DEPART :
DEPART
21231    1667
21617     622
21054     510
21603     396
25056     294
71076     289
21166     266
21425     224
21154     188
21515     184
Name: count, dtype: int64

Top 10 codes postaux les plus fréquents pour ARRIVEE :
ARRIVEE
21231    1634
21617     635
21054     509
21603     428
71076     287
21166     282
25056     256
21154     204
21425     202
21515     198
Name: count, dtype: int64


In [6]:
total_rows = df.shape[0]

depart_top_10 = df['DEPART'].value_counts().head(10)
arrivee_top_10 = df['ARRIVEE'].value_counts().head(10)

total_depart_appearances = df['DEPART'].value_counts().sum()
total_arrivee_appearances = df['ARRIVEE'].value_counts().sum()

print(f"Nombre total de lignes : {total_rows}")
print("\nTop 10 codes postaux les plus fréquents pour DEPART :")
print(depart_top_10)
print("\nTop 10 codes postaux les plus fréquents pour ARRIVEE :")
print(arrivee_top_10)
print(f"\nNombre total d'apparitions pour DEPART : {total_depart_appearances}")
print(f"Nombre total d'apparitions pour ARRIVEE : {total_arrivee_appearances}")

Nombre total de lignes : 18557

Top 10 codes postaux les plus fréquents pour DEPART :
DEPART
21231    1667
21617     622
21054     510
21603     396
25056     294
71076     289
21166     266
21425     224
21154     188
21515     184
Name: count, dtype: int64

Top 10 codes postaux les plus fréquents pour ARRIVEE :
ARRIVEE
21231    1634
21617     635
21054     509
21603     428
71076     287
21166     282
25056     256
21154     204
21425     202
21515     198
Name: count, dtype: int64

Nombre total d'apparitions pour DEPART : 18557
Nombre total d'apparitions pour ARRIVEE : 18557


In [7]:
combinations_freq = df.groupby(['DEPART', 'ARRIVEE']).size().reset_index(name='count')

top_combinations = combinations_freq.sort_values(by='count', ascending=False).head(10)

print("Top 10 combinaisons les plus fréquentes entre DEPART et ARRIVEE :")
print(top_combinations)

Top 10 combinaisons les plus fréquentes entre DEPART et ARRIVEE :
       DEPART  ARRIVEE  count
3806    21231    21231     11
4167    21231    21599      5
5016    21278    21231      5
0           0        0      4
6497    21416    21231      4
10591   25056    21572      4
4153    21231    21584      4
3221    21189    21231      4
4151    21231    21582      4
3822    21231    21247      4


In [11]:
df_code_postaux = pd.read_excel("codePostauxBourgogneFrancheComte.xlsx")
df_couples = pd.read_csv("couples_depart_arrivee.csv")

# Renommer la colonne Code INSEE pour correspondre aux colonnes DEPART et ARRIVEE
df_code_postaux = df_code_postaux.rename(columns={"Code INSEE": "INSEE"})

# Merge pour trouver les détails des codes postaux pour chaque couple de départ et arrivée
df_merged_depart = df_couples.merge(df_code_postaux, left_on='DEPART', right_on='INSEE', how='left')
df_merged_depart = df_merged_depart.rename(columns={'Code Postal': 'Code Postal DEPART'}).drop('INSEE', axis=1)

df_merged_arrivee = df_merged_depart.merge(df_code_postaux, left_on='ARRIVEE', right_on='INSEE', how='left')
df_merged_arrivee = df_merged_arrivee.rename(columns={'Code Postal': 'Code Postal ARRIVEE'}).drop('INSEE', axis=1)

# Filtrer le résultat pour n'avoir que les colonnes souhaitées avec toutes les informations
filtered_df = df_merged_arrivee

# Sauvegarde du fichier filtré en Excel
filtered_df.to_excel("filtered_code_postaux_paires.xlsx", index=False)

print("Le fichier filtré a été sauvegardé sous le nom 'filtered_code_postaux_paires.xlsx'")

Le fichier filtré a été sauvegardé sous le nom 'filtered_code_postaux_paires.xlsx'


In [ ]:
import openrouteservice
import time

client = openrouteservice.Client(key='5b3ce3597851110001cf6248a2d67d3b0a9e4917b11f1e83ab06a71c')

file_path = 'filtered_code_postaux_paires.xlsx'
df_filtered = pd.read_excel(file_path)


df_filtered['geo_point_2d DEPART'] = df_filtered['geo_point_2d DEPART'].apply(lambda x: tuple(map(float, x.split(','))))
df_filtered['geo_point_2d ARRIVEE'] = df_filtered['geo_point_2d ARRIVEE'].apply(lambda x: tuple(map(float, x.split(','))))

# obtenir la distance via l'API
def get_distance(coord1, coord2):
    try:
        routes = client.directions([coord1, coord2])
        distance = routes['routes'][0]['summary']['distance']  # en mètres
        return distance / 1000  # Convertir en km
    except Exception as e:
        print(f"Erreur lors du calcul de la distance entre {coord1} et {coord2}: {e}")
        return None

# Calculer la distance pour chaque paire 
distances = []
for idx, row in df_filtered.iterrows():
    coord_depart = row['geo_point_2d DEPART']
    coord_arrivee = row['geo_point_2d ARRIVEE']
    distance = get_distance(coord_depart, coord_arrivee)
    distances.append(distance)
    time.sleep(1)  # Pause pour éviter de dépasser le quota API


df_filtered['distance_en_km'] = distances

df_filtered.to_excel('filtered_code_postaux_with_distances.xlsx', index=False)

print("Le fichier avec les distances a été sauvegardé sous le nom 'filtered_code_postaux_with_distances.xlsx'")